In [2]:
import numpy as np
import scanpy as sc
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score


In [3]:
def subset_anndata(ad: sc.AnnData, n_cells: int, random_state: int = 0) -> sc.AnnData:
    rng = np.random.default_rng(random_state)
    n = ad.n_obs
    n_subset = min(n_cells, n)
    idx = rng.choice(np.arange(n), size=n_subset, replace=False)
    return ad[idx].copy()


def compute_ari_fixed(X, labels, k, random_state: int = 0) -> float:
    Xk = X[:, :k]
    km = KMeans(
        n_clusters=len(np.unique(labels)),
        n_init=10,
        random_state=random_state,
    )
    km.fit(Xk)
    return adjusted_rand_score(labels, km.labels_)


In [4]:
def make_layer_schedule(
    pattern_type: str,
    n_layers: int,
    t_euclid: float,
) -> list[dict]:
    """
    Returns a list of layer specs:
    [
      {"metric_mode": "scm", "hessian_mix_mode": None, "t": 2.0},
      {"metric_mode": "euclidean", "hessian_mix_mode": None, "t": t_euclid},
      ...
    ]
    """
    layers = []

    for ell in range(n_layers):
        if pattern_type == "eucl_only":
            layers.append(
                dict(metric_mode="euclidean", hessian_mix_mode=None, t=t_euclid)
            )

        elif pattern_type == "scm_alt_euclid":
            if ell % 2 == 0:  # even layers = SCM
                layers.append(
                    dict(metric_mode="scm", hessian_mix_mode=None, t=2.0)
                )
            else:
                layers.append(
                    dict(metric_mode="euclidean", hessian_mix_mode=None, t=t_euclid)
                )

        elif pattern_type == "hessNone_alt_euclid":
            if ell % 2 == 0:
                layers.append(
                    dict(
                        metric_mode="hessian_mixed",
                        hessian_mix_mode="none",
                        t=2.0,
                    )
                )
            else:
                layers.append(
                    dict(metric_mode="euclidean", hessian_mix_mode=None, t=t_euclid)
                )

        elif pattern_type == "hessMult_alt_euclid":
            if ell % 2 == 0:
                layers.append(
                    dict(
                        metric_mode="hessian_mixed",
                        hessian_mix_mode="multiplicative",
                        t=1.0,
                    )
                )
            else:
                layers.append(
                    dict(metric_mode="euclidean", hessian_mix_mode=None, t=t_euclid)
                )

        else:
            raise ValueError(f"Unknown pattern_type: {pattern_type!r}")

    return layers


In [5]:
import numpy as np
import pandas as pd
import copy
from scipy.sparse.linalg import ArpackNoConvergence

from scripts.EGGFM.train_energy import train_energy_model
from scripts.EGGFM.engine import EGGFMDiffusionEngine  # or whatever you named it


def run_layered_admr_config_subset(
    base_ad: sc.AnnData,
    base_params: dict,
    *,
    exp_name: str,
    pattern_type: str,      # "eucl_only", "scm_alt_euclid", ...
    n_layers: int,
    t_euclid: float,
    norm_type: str,
    distance_power: float,
    n_runs: int = 5,
    n_cells_sample: int = 1000,
    base_seed: int = 0,
) -> pd.DataFrame:
    """
    True layered ADMR ablation:
      - trains ONE energy model per run
      - stacks n_layers diffusion layers with *different metrics* per layer
      - uses previous embedding as geometry for next layer
    """
    print(
        f"\n========== {exp_name} ==========\n"
        f"pattern_type={pattern_type}, n_layers={n_layers}, "
        f"t_euclid={t_euclid}, norm_type={norm_type}, distance_power={distance_power}"
    )

    params = copy.deepcopy(base_params)
    diff_cfg = params["eggfm_diffmap"]
    train_cfg = params["eggfm_train"]

    # global norm / power for this config
    diff_cfg["norm_type"] = norm_type
    diff_cfg["distance_power"] = float(distance_power)

    spec = params["spec"]
    k = spec.get("ari_n_dims", spec.get("n_pcs", 10))
    label_key = spec["ari_label_key"]

    records: list[dict] = []

    for run in range(n_runs):
        run_seed = base_seed + run
        print(
            f"\n=== {exp_name} | Run {run+1}/{n_runs} (seed={run_seed}) ===",
            flush=True,
        )

        # 0) Subset + prep
        ad_prep = subset_anndata(base_ad, n_cells_sample, random_state=run_seed)

        # 1) Train energy model ONCE per run
        energy_model = train_energy_model(ad_prep, params["eggfm_model"], train_cfg)

        # 2) Instantiate diffusion engine
        engine = EGGFMDiffusionEngine(energy_model=energy_model, diff_cfg=diff_cfg)

        labels = ad_prep.obs[label_key].to_numpy()

        # 3) Build layer schedule for this config
        layer_specs = make_layer_schedule(
            pattern_type=pattern_type,
            n_layers=n_layers,
            t_euclid=t_euclid,
        )

        # geometry for layer 0: PCA (or whatever geometry_source says)
        X_geom_prev = None  # None => engine uses geometry_source in diff_cfg ("pca")

        for layer_idx, layer_spec in enumerate(layer_specs):
            metric_mode = layer_spec["metric_mode"]
            h_mode = layer_spec["hessian_mix_mode"]
            t_layer = layer_spec["t"]

            # wire up metric-specific bits
            diff_cfg["metric_mode"] = metric_mode
            if metric_mode == "hessian_mixed":
                diff_cfg["hessian_mix_mode"] = h_mode

            layer_label = (
                "ADMR_L0"
                if layer_idx == 0
                else f"ADMR_L{layer_idx}"
            )

            try:
                # Build embedding for this layer
                X_emb = engine.build_embedding(
                    ad_prep=ad_prep,
                    metric_mode=metric_mode,
                    t_override=t_layer,
                    X_geom_override=X_geom_prev,   # None for L0, prev emb for L>0
                )  # shape (n_cells, n_comps)

                # store for potential downstream use
                ad_prep.obsm[layer_label] = X_emb
                X_geom_prev = X_emb  # next layer uses this geometry

                # ARI on first k dims
                Xk = X_emb[:, :k]
                ari = compute_ari_fixed(Xk, labels, k, random_state=run_seed)
                print(
                    f"  Layer {layer_idx} ({layer_label}): "
                    f"metric={metric_mode}, h_mode={h_mode}, t={t_layer} → ARI={ari:.4f}"
                )

                records.append(
                    dict(
                        exp_name=exp_name,
                        run=run,
                        pattern_type=pattern_type,
                        layer=layer_idx,
                        layer_label=layer_label,
                        metric_mode=metric_mode,
                        hessian_mix_mode=h_mode,
                        t=t_layer,
                        norm_type=norm_type,
                        distance_power=distance_power,
                        n_cells=ad_prep.n_obs,
                        ari=ari,
                        note=None,
                    )
                )

            except ArpackNoConvergence as e:
                print(
                    f"  [WARN] ARPACK failed at layer {layer_idx} "
                    f"(metric={metric_mode}, h_mode={h_mode}): {e}"
                )
                records.append(
                    dict(
                        exp_name=exp_name,
                        run=run,
                        pattern_type=pattern_type,
                        layer=layer_idx,
                        layer_label=layer_label,
                        metric_mode=metric_mode,
                        hessian_mix_mode=h_mode,
                        t=t_layer,
                        norm_type=norm_type,
                        distance_power=distance_power,
                        n_cells=ad_prep.n_obs,
                        ari=np.nan,
                        note="ArpackNoConvergence",
                    )
                )
                break  # stop deeper layers for this run

    df = pd.DataFrame.from_records(records)

    print(f"\n--- Summary: {exp_name} ---")
    summary = (
        df.groupby("layer_label")["ari"]
        .agg(["mean", "std"])
        .sort_index()
    )
    display(summary)

    return df


In [7]:
params = {
    "seed": 7,
    "pca_n_top_genes": 2000,

    "spec": {
        "n_pcs": 20,
        "dcol_max_cells": 3000,
        "ari_label_key": "Cell type annotation",  # <-- this must match an obs column name
        # "ari_label_key": "paul15_clusters",         # <-- this must match an obs column name
        "ari_n_dims": 10,                           # how many dims to use for ARI per embedding
        # "ad_file": "data/paul15/paul15.h5ad",
        "ad_file": "data/prep/qc.h5ad",
    },

    "qc": {
        "min_cells": 500,
        "min_genes": 200,
        "max_pct_mt": 15,
    },

    "eggfm_model": {
        "hidden_dims": [512, 512, 512, 512],
    },

    "eggfm_train": {
        "batch_size": 2048,
        "num_epochs": 50,
        "lr": 1.0e-4,
        "sigma": 0.1,
        "device": "cuda",
        "latent_space": "hvg",
        "early_stop_patience": 10,
        "early_stop_min_delta": 0.0,
        "n_cells_sample":2000
    },

    "eggfm_diffmap": {
        "geometry_source": "pca",          # "pca" or "hvg"
        "energy_source": "hvg",            # where SCM/Hessian read energies
        "metric_mode": "scm",    # "euclidean", "scm", or "hessian_mixed"
        "n_neighbors": 30,
        "n_comps": 30,
        "device": "cuda",
        "hvp_batch_size": 1024,
        "eps_mode": "median",
        "eps_value": 1.0,
        "eps_trunc": "no",
        "distance_power": 1,
        "t": 1.5,
        "norm_type": "linf",

        # SCM hypered
        "metric_gamma": 0.2,
        "metric_lambda": 4.0,
        "energy_clip_abs": 3.0,
        "energy_batch_size": 2048,

        # Hessian mixing hyperparams
        "hessian_mix_mode": "multiplicative",   # "additive" | "multiplicative" | "none"
        "hessian_mix_alpha": 0.3,
        "hessian_beta": 0.2,
        "hessian_clip_std": 2.0,
        "hessian_use_neg": True,
    },
}
base = sc.read_h5ad(params["spec"]["ad_file"])

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:


patterns = [ ("eucl_only", 4), ("scm_alt_euclid", 4), ("hessNone_alt_euclid", 4), ("hessMult_alt_euclid", 4), ] 
norm_types = ["l2", "l1", "linf", "l0"] 
distance_powers = [0.0, 1.0, 2.0] 
t_euclid_values = [1.0, 2.0] 
config_list = [] 
for pattern_type, n_layers in patterns: 
    for norm in norm_types: 
        for p in distance_powers: 
            for t_eucl in t_euclid_values: 
                exp_name = ( f"{pattern_type}_L{n_layers}_norm{norm}_p{p}_teucl{t_eucl}" ) 
                config_list.append( dict( exp_name=exp_name, pattern_type=pattern_type, n_layers=n_layers, t_euclid=t_eucl, norm_type=norm, distance_power=p, ) )
all_results = []
for i, cfg in enumerate(config_list):
    print(f"\n\n######## Config {i+1}/{len(config_list)}: {cfg['exp_name']} ########")
    df_cfg = run_layered_admr_config_subset(
        base_ad=base,
        base_params=params,
        n_runs=5,
        n_cells_sample=1000,
        base_seed=123,
        **cfg,
    )
    all_results.append(df_cfg)

results_df = pd.concat(all_results, ignore_index=True)
results_path = "out/eggfm_admr_layered_ablation_subset1000.csv"
results_df.to_csv(results_path, index=False)

import os
gcs_path = f"gs://medit-uml-prod-uscentral1-8e7a/{results_path}"
os.system(f"gsutil cp {results_path} {gcs_path}")
print("Uploaded to:", gcs_path)




######## Config 1/96: eucl_only_L4_norml2_p0.0_teucl1.0 ########

========== eucl_only_L4_norml2_p0.0_teucl1.0 ==========
pattern_type=eucl_only, n_layers=4, t_euclid=1.0, norm_type=l2, distance_power=0.0

=== eucl_only_L4_norml2_p0.0_teucl1.0 | Run 1/5 (seed=123) ===


/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] Epoch 2/50  loss=0.0000
[Energy DSM] Epoch 3/50  loss=0.0000
[Energy DSM] Epoch 4/50  loss=0.0000
[Energy DSM] Epoch 5/50  loss=0.0000
[Energy DSM] Epoch 6/50  loss=0.0000
[Energy DSM] Epoch 7/50  loss=0.0000
[Energy DSM] Epoch 8/50  loss=0.0000
[Energy DSM] Epoch 9/50  loss=0.0000
[Energy DSM] Epoch 10/50  loss=0.0000
[Energy DSM] Epoch 11/50  loss=0.0000
[Energy DSM] Early stopping at epoch 11 (best_loss=0.0000)
[AnnDataViewProvider] using PCA for geometry with shape (1000, 20)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 0 (ADMR_L0): metric=euclidean, h_mode=None, t=1.0 → ARI=0.3021
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] to

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 1 (ADMR_L1): metric=euclidean, h_mode=None, t=1.0 → ARI=0.2560
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 2 (ADMR_L2): metric=euclidean, h_mode=None, t=1.0 → ARI=0.4075
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 3 (ADMR_L3): metric=euclidean, h_mode=None, t=1.0 → ARI=0.3462

=== eucl_only_L4_norml2_p0.0_teucl1.0 | Run 3/5 (seed=125) ===
[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] 

...

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


  [EGGFM Metrics] batch 30/30 (30000/30000 edges)
[DiffusionMap] using eps = 260 (power p=2.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 0 (ADMR_L0): metric=hessian_mixed, h_mode=multiplicative, t=1.0 → ARI=0.2592
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 900 (power p=2.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 1 (ADMR_L1): metric=euclidean, h_mode=None, t=2.0 → ARI=0.2687
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[EGGFM Metrics] computing Hessian-mixed edge lengths...
  [EGGFM Metrics] batch 30/30 (30000/30000 edges)
[DiffusionMap] using eps = 265.5 (power p=2.0)
[DiffusionMap] co

,mean,std
layer_label,,
ADMR_L0,0.247818,0.017928
ADMR_L1,0.258195,0.035171
ADMR_L2,0.269562,0.063458
ADMR_L3,0.030582,0.083347


Copying file://out/eggfm_admr_layered_ablation_subset1000.csv [Content-Type=text/csv]...
/ [1 files][224.3 KiB/224.3 KiB]                                                
Operation completed over 1 objects/224.3 KiB.                                    


Uploaded to: gs://medit-uml-prod-uscentral1-8e7a/out/eggfm_admr_layered_ablation_subset1000.csv


In [6]:
# Focused hyperparam grid (with SCM kept, but targeted)

# For each pattern_type we can choose its own depth + distance_power grid
pattern_grid = {
    "eucl_only": {
        "n_layers": [1, 2, 3, 4,5,6,7,8],
        "distance_powers": [0.0, 0.25, 0.5, 0.75, 1.0],
    },
    "hessMult_alt_euclid": {
                "n_layers": [1, 2, 3, 4,5,6,7,8],

        "distance_powers": [0.0, 0.25, 0.5, 0.75, 1.0],
    },
    # SCM: concentrate where we already saw it behave well
    "scm_alt_euclid": {
        "n_layers": [1, 2, 3, 4,5,6,7,8],
        "distance_powers": [0.0, 0.25, 0.5],
    },
}

# Lock in best norm + t_euclid from the first sweep
norm_types = ["l0"]
t_euclid_values = [2.0]

config_list = []
for pattern_type, grid in pattern_grid.items():
    for n_layers in grid["n_layers"]:
        for p in grid["distance_powers"]:
            for norm in norm_types:
                for t_eucl in t_euclid_values:
                    exp_name = (
                        f"{pattern_type}_L{n_layers}_norm{norm}_p{p}_teucl{t_eucl}"
                    )
                    config_list.append(
                        dict(
                            exp_name=exp_name,
                            pattern_type=pattern_type,
                            n_layers=n_layers,
                            t_euclid=t_eucl,
                            norm_type=norm,
                            distance_power=p,
                        )
                    )

len(config_list)


104

In [8]:
import os
from datetime import datetime
import pandas as pd
pattern_grid = {
    # Euclidean baseline: test both l0 and linf, since linf was a star on Paul15
    "eucl_only": {
        "n_layers": [1, 2, 3],
        "norm_types": ["l0", "linf"],
        "distance_powers": [0.0, 0.25, 0.5],
    },
    # SCM: winner on Weinreb at L3, p ~ 0.25, norm l0
    "scm_alt_euclid": {
        "n_layers": [1, 2, 3],
        "norm_types": ["l0"],
        "distance_powers": [0.0, 0.25, 0.5],
    },
    # Hessian-mixed: keep as a single EGGFM competitor
    "hessMult_alt_euclid": {
        "n_layers": [1, 2, 3],
        "norm_types": ["l0"],
        "distance_powers": [0.25, 0.5],
    },
}

t_euclid_values = [2.0]

config_list = []
for pattern_type, grid in pattern_grid.items():
    for n_layers in grid["n_layers"]:
        for norm in grid["norm_types"]:
            for p in grid["distance_powers"]:
                for t_eucl in t_euclid_values:
                    exp_name = (
                        f"{pattern_type}_L{n_layers}_norm{norm}_p{p}_teucl{t_eucl}"
                    )
                    config_list.append(
                        dict(
                            exp_name=exp_name,
                            pattern_type=pattern_type,
                            n_layers=n_layers,
                            t_euclid=t_eucl,
                            norm_type=norm,
                            distance_power=p,
                        )
                    )

len(config_list)


all_results = []
for i, cfg in enumerate(config_list):
    print(f"\n\n######## Config {i+1}/{len(config_list)}: {cfg['exp_name']} ########")
    df_cfg = run_layered_admr_config_subset(
        base_ad=base,
        base_params=params,
        n_runs=5,
        n_cells_sample=1000,
        base_seed=123,
        **cfg,
    )
    all_results.append(df_cfg)

results_df = pd.concat(all_results, ignore_index=True)

# ---- dynamic path bits ----
results_dir = "out"
os.makedirs(results_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# pull some global identifiers from params
ari_label_key = params["spec"]["ari_label_key"]
label_slug = ari_label_key.replace(" ", "_")
n_cells_sample = params["eggfm_train"]["n_cells_sample"]

filename = f"eggfm_admr_layered_ablation_cells{n_cells_sample}_{label_slug}_{timestamp}.csv"
results_path = os.path.join(results_dir, filename)

results_df.to_csv(results_path, index=False)

gcs_path = f"gs://medit-uml-prod-uscentral1-8e7a/{results_path}"
os.system(f"gsutil cp {results_path} {gcs_path}")
print("Uploaded to:", gcs_path)




######## Config 1/33: eucl_only_L1_norml0_p0.0_teucl2.0 ########

========== eucl_only_L1_norml0_p0.0_teucl2.0 ==========
pattern_type=eucl_only, n_layers=1, t_euclid=2.0, norm_type=l0, distance_power=0.0

=== eucl_only_L1_norml0_p0.0_teucl2.0 | Run 1/5 (seed=123) ===
[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] Epoch 2/50  loss=0.0000
[Energy DSM] Epoch 3/50  loss=0.0000
[Energy DSM] Epoch 4/50  loss=0.0000
[Energy DSM] Epoch 5/50  loss=0.0000
[Energy DSM] Epoch 6/50  loss=0.0000
[Energy DSM] Epoch 7/50  loss=0.0000
[Energy DSM] Epoch 8/50  loss=0.0000
[Energy DSM] Epoch 9/50  loss=0.0000
[Energy DSM] Epoch 10/50  loss=0.0000
[Energy DSM] Epoch 11/50  loss=0.0000
[Energy DSM] Early stopping at epoch 11 (best_loss=0.0000)
[AnnDataViewProvider] using PCA for geometry with shape (1000, 20)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors..

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] Epoch 2/50  loss=0.0000
[Energy DSM] Epoch 3/50  loss=0.0000
[Energy DSM] Epoch 4/50  loss=0.0000
[Energy DSM] Epoch 5/50  loss=0.0000
[Energy DSM] Epoch 6/50  loss=0.0000
[Energy DSM] Epoch 7/50  loss=0.0000
[Energy DSM] Epoch 8/50  loss=0.0000
[Energy DSM] Epoch 9/50  loss=0.0000
[Energy DSM] Epoch 10/50  loss=0.0000
[Energy DSM] Epoch 11/50  loss=0.0000
[Energy DSM] Early stopping at epoch 11 (best_loss=0.0000)
[AnnDataViewProvider] using PCA for geometry with shape (1000, 20)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 0 (ADMR_L0): metric=euclidean, h_mode=None, t=2.0 → ARI=0.2385

=== eucl_only_L1_norml0_p0.0_teucl2.0 | Run 4/5 (seed=126) ===
[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] Epoch 2/50  loss=0.0000
[Energy

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


[Energy DSM] Epoch 1/50  loss=0.0000
[Energy DSM] Epoch 2/50  loss=0.0000
[Energy DSM] Epoch 3/50  loss=0.0000
[Energy DSM] Epoch 4/50  loss=0.0000
[Energy DSM] Epoch 5/50  loss=0.0000
[Energy DSM] Epoch 6/50  loss=0.0000
[Energy DSM] Epoch 7/50  loss=0.0000
[Energy DSM] Epoch 8/50  loss=0.0000
[Energy DSM] Epoch 9/50  loss=0.0000
[Energy DSM] Epoch 10/50  loss=0.0000
[Energy DSM] Epoch 11/50  loss=0.0000
[Energy DSM] Early stopping at epoch 11 (best_loss=0.0000)
[AnnDataViewProvider] using PCA for geometry with shape (1000, 20)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 1 (power p=0.0)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 0 (ADMR_L0): metric=euclidean, h_mode=None, t=2.0 → ARI=0.2699

--- Summary: eucl_only_L1_norml0_p0.0_teucl2.0 ---


...

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


  [EGGFM Metrics] batch 30/30 (30000/30000 edges)
[DiffusionMap] using eps = 4.019 (power p=0.5)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 0 (ADMR_L0): metric=hessian_mixed, h_mode=multiplicative, t=1.0 → ARI=0.2692
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[DiffusionMap] using eps = 5.477 (power p=0.5)
[DiffusionMap] computing eigenvectors...
[DiffusionMap] finished. Embedding shape: (1000, 30)
  Layer 1 (ADMR_L1): metric=euclidean, h_mode=None, t=2.0 → ARI=0.2689
[EGGFM Engine] using override geometry with shape (1000, 30)
[EGGFM Engine] building kNN graph (euclidean in geometry space)...
[EGGFM Engine] total edges (directed): 30000
[EGGFM Metrics] computing Hessian-mixed edge lengths...
  [EGGFM Metrics] batch 30/30 (30000/30000 edges)
[DiffusionMap] using eps = 4.048 (power p=0.5)
[DiffusionMap

,mean,std
layer_label,,
ADMR_L0,0.261980,0.028368
ADMR_L1,0.289298,0.068476
ADMR_L2,0.313157,0.047459


Copying file://out/eggfm_admr_layered_ablation_cells2000_Cell_type_annotation_20251128_221309.csv [Content-Type=text/csv]...
/ [1 files][ 36.4 KiB/ 36.4 KiB]                                                
Operation completed over 1 objects/36.4 KiB.                                     


Uploaded to: gs://medit-uml-prod-uscentral1-8e7a/out/eggfm_admr_layered_ablation_cells2000_Cell_type_annotation_20251128_221309.csv
